# Gather Historic Data
This notebook is, for now, more of experimental work. I want to gather multiple sources of data - preferably automate this process. For starters I want to gather historical data since they are more of a one time job.

## Start with all results from the first BL

I found multiple websites to gather. I took this one that is the simplest to scrap. Since no `robot.txt` is present I dont think this is a problem, because I only want to gather this one time only.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# List to store scraped data
data = []

# Iterate over the pages (from 101 to 161 inclusive)
for i in range(101, 162):
    url = f"http://www.bulibox.de/spieltage/B100{i}.html"
    print(f"Scraping: {url}")
    response = requests.get(url)
    
    # Check if the page was retrieved successfully
    if response.status_code != 200:
        print(f"Error retrieving {url}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")
    
    # Get season info from the <h4> tag (if available)
    h4_tag = soup.find("h4")
    season = h4_tag.text.strip() if h4_tag else "Unknown Season"
    
    # Find all matchday markers (<b class="bulired">)
    matchday_tags = soup.find_all("b", class_="bulired")
    
    for matchday_tag in matchday_tags:
        matchday = matchday_tag.text.strip()
        # Get the table that immediately follows this matchday header
        table = matchday_tag.find_next("table")
        if table:
            rows = table.find_all("tr")[1:]  # skip header row
            for row in rows:
                cells = row.find_all("td")
                if len(cells) >= 3:
                    spielpaarung = cells[0].get_text(strip=True)
                    ergebnis = cells[1].get_text(strip=True)
                    datum = cells[2].get_text(strip=True)
                    
                    data.append({
                        "Season": season,
                        "Spieltag": matchday,
                        "Spielpaarung": spielpaarung,
                        "Ergebnis": ergebnis,
                        "Datum": datum
                    })
    
    # Wait 5 seconds before the next request to be compliant with the site's usage
    time.sleep(5)

# Create a DataFrame from the collected data
df = pd.DataFrame(data)


Scraping: http://www.bulibox.de/spieltage/B100101.html
Scraping: http://www.bulibox.de/spieltage/B100102.html
Scraping: http://www.bulibox.de/spieltage/B100103.html
Scraping: http://www.bulibox.de/spieltage/B100104.html
Scraping: http://www.bulibox.de/spieltage/B100105.html
Scraping: http://www.bulibox.de/spieltage/B100106.html
Scraping: http://www.bulibox.de/spieltage/B100107.html
Scraping: http://www.bulibox.de/spieltage/B100108.html
Scraping: http://www.bulibox.de/spieltage/B100109.html
Scraping: http://www.bulibox.de/spieltage/B100110.html
Scraping: http://www.bulibox.de/spieltage/B100111.html
Scraping: http://www.bulibox.de/spieltage/B100112.html
Scraping: http://www.bulibox.de/spieltage/B100113.html
Scraping: http://www.bulibox.de/spieltage/B100114.html
Scraping: http://www.bulibox.de/spieltage/B100115.html
Scraping: http://www.bulibox.de/spieltage/B100116.html
Scraping: http://www.bulibox.de/spieltage/B100117.html
Scraping: http://www.bulibox.de/spieltage/B100118.html
Scraping: 

In [2]:
df.to_csv("data/first/first_bl_results.csv")

## Gather all tables


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# Directory to store CSV files
output_dir = "data/first/abschlusstabellen"
os.makedirs(output_dir, exist_ok=True)

# Iterate over pages from 101 to 161 inclusive
for i in range(101, 162):
    url = f"http://www.bulibox.de/abschlusstabellen/B100{i}.html"
    print(f"Scraping: {url}")
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error retrieving {url}")
        continue
    
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="abschluss")
    if not table:
        print(f"No table found at {url}")
        continue

    # Parse table header
    header_row = table.find("tr")
    headers = [th.get_text(strip=True) for th in header_row.find_all("th")]
    
    # Parse table rows
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = tr.find_all("td")
        row = []
        for cell in cells:
            # If the cell contains a link, get its text; otherwise, use cell text
            a_tag = cell.find("a")
            cell_text = a_tag.get_text(strip=True) if a_tag else cell.get_text(strip=True)
            row.append(cell_text)
        rows.append(row)
    
    # Create DataFrame from the table data
    df_table = pd.DataFrame(rows, columns=headers)
    
    # Extract season from the "Statistik" column of the first row, e.g., "Saison 2023/2024"
    if "Statistik" in df_table.columns and not df_table.empty:
        season_text = df_table.loc[0, "Statistik"]
        season = season_text.replace("Saison", "").strip()
        # Replace slashes with dashes to avoid directory issues (e.g., "2023/2024" -> "2023-2024")
        season = season.replace("/", "-")
    else:
        season = f"season_{i}"
    
    # Build output path and save DataFrame as CSV
    output_path = os.path.join(output_dir, f"{season}.csv")
    df_table.to_csv(output_path, index=False)
    print(f"Saved table for season {season} to {output_path}")
    
    # Wait 5 seconds before the next request
    time.sleep(5)


Scraping: http://www.bulibox.de/abschlusstabellen/B100101.html
Saved table for season 1963-1964 to data/abschlusstabellen\1963-1964.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B100102.html
Saved table for season 1964-1965 to data/abschlusstabellen\1964-1965.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B100103.html
Saved table for season 1965-1966 to data/abschlusstabellen\1965-1966.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B100104.html
Saved table for season 1966-1967 to data/abschlusstabellen\1966-1967.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B100105.html
Saved table for season 1967-1968 to data/abschlusstabellen\1967-1968.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B100106.html
Saved table for season 1968-1969 to data/abschlusstabellen\1968-1969.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B100107.html
Saved table for season 1969-1970 to data/abschlusstabellen\1969-1970.csv
Scraping: http://www.bulibox.de/abschlusstabelle

## Gasther all results from second bl
This is mandatory since each team can drop to the second BL. I will gather all results from the second BL as well.

In [ ]:
# Base URL of the main page
base_url = "http://www.bulibox.de/abschlusstabellen/index-2.liga.html"

# List to store scraped data
data = []

# Fetch the main page
response = requests.get(base_url)
if response.status_code != 200:
    print(f"Error retrieving {base_url}")
    exit()

main_soup = BeautifulSoup(response.content, "html.parser")

# Extract all rows from the table
rows = main_soup.find_all("tr")

# Iterate over each row to find season data
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 4:
        # Extract the season description
        season_desc = cells[0].get_text(strip=True)
        
        # Extract the 'Ergebnisse' link
        ergebnisse_link = cells[2].find("a", href=True)
        if ergebnisse_link:
            relative_url = ergebnisse_link['href']
            # Construct the full URL for the 'Ergebnisse' page
            season_url = "http://www.bulibox.de" + relative_url[2:]  # Adjusting the relative URL
            print(f"Scraping: {season_url}")
            
            season_response = requests.get(season_url)
            if season_response.status_code != 200:
                print(f"Error retrieving {season_url}")
                continue

            season_soup = BeautifulSoup(season_response.content, "html.parser")

            # Get season info from the <h4> tag (if available)
            h4_tag = season_soup.find("h4")
            season = h4_tag.text.strip() if h4_tag else season_desc

            # Find all matchday markers (<b class="bulired">)
            matchday_tags = season_soup.find_all("b", class_="bulired")

            for matchday_tag in matchday_tags:
                matchday = matchday_tag.text.strip()
                # Get the table that immediately follows this matchday header
                table = matchday_tag.find_next("table")
                if table:
                    rows = table.find_all("tr")[1:]  # Skip header row
                    for row in rows:
                        cells = row.find_all("td")
                        if len(cells) >= 3:
                            spielpaarung = cells[0].get_text(strip=True)
                            ergebnis = cells[1].get_text(strip=True)
                            datum = cells[2].get_text(strip=True)
                            
                            data.append({
                                "Season": season,
                                "Spieltag": matchday,
                                "Spielpaarung": spielpaarung,
                                "Ergebnis": ergebnis,
                                "Datum": datum
                            })

            # Wait 5 seconds before the next request to be compliant with the site's usage policy
            time.sleep(5)

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Save to CSV for later analysis
df.to_csv("data/second/second_bl_results.csv", index=False)

df


Scraping: http://www.bulibox.de/spieltage/B200250.html
Scraping: http://www.bulibox.de/spieltage/B200249.html
Scraping: http://www.bulibox.de/spieltage/B200248.html
Scraping: http://www.bulibox.de/spieltage/B200247.html
Scraping: http://www.bulibox.de/spieltage/B200246.html
Scraping: http://www.bulibox.de/spieltage/B200245.html
Scraping: http://www.bulibox.de/spieltage/B200244.html
Scraping: http://www.bulibox.de/spieltage/B200243.html
Scraping: http://www.bulibox.de/spieltage/B200242.html
Scraping: http://www.bulibox.de/spieltage/B200241.html
Scraping: http://www.bulibox.de/spieltage/B200240.html
Scraping: http://www.bulibox.de/spieltage/B200239.html
Scraping: http://www.bulibox.de/spieltage/B200238.html
Scraping: http://www.bulibox.de/spieltage/B200237.html
Scraping: http://www.bulibox.de/spieltage/B200236.html
Scraping: http://www.bulibox.de/spieltage/B200235.html
Scraping: http://www.bulibox.de/spieltage/B200234.html
Scraping: http://www.bulibox.de/spieltage/B200233.html
Scraping: 

,Season,Spieltag,Spielpaarung,Ergebnis,Datum
0,2. Bundesliga - Saison 2023/2024: Alle Spiele,1. Spieltag,Hamburger SV - FC Schalke 04,5:3,28.07.2023
1,2. Bundesliga - Saison 2023/2024: Alle Spiele,1. Spieltag,1. FC Kaiserslautern - FC St. Pauli,1:2,29.07.2023
2,2. Bundesliga - Saison 2023/2024: Alle Spiele,1. Spieltag,Hannover 96 - SV Elversberg,2:2,29.07.2023
3,2. Bundesliga - Saison 2023/2024: Alle Spiele,1. Spieltag,VfL Osnabrück - Karlsruher SC,2:3,29.07.2023
4,2. Bundesliga - Saison 2023/2024: Alle Spiele,1. Spieltag,SV Wehen Wiesbaden - 1. FC Magdeburg,1:1,29.07.2023
...,...,...,...,...,...
19613,2. Bundesliga SÜD - Saison 1974/1975,38. Spieltag,Wormatia Worms - 1. FSV Mainz 05,2:3,15.06.1975
19614,2. Bundesliga SÜD - Saison 1974/1975,38. Spieltag,1. FC Saarbrücken - Stuttgarter Kickers,1:1,15.06.1975
19615,2. Bundesliga SÜD - Saison 1974/1975,38. Spieltag,FC Schweinfurt 05 - Chio Waldhof 07,2:0,15.06.1975
19616,2. Bundesliga SÜD - Saison 1974/1975,38. Spieltag,FC Homburg - SpVgg Bayreuth,1:0,15.06.1975


## Gather second liga abschlusstabellen

In [7]:
#Base URL of the main page
base_url = "http://www.bulibox.de/abschlusstabellen/index-2.liga.html"

# Directory to store CSV files
output_dir = "data/second/abschlusstabellen"
os.makedirs(output_dir, exist_ok=True)

# Fetch the main page
response = requests.get(base_url)
if response.status_code != 200:
    print(f"Error retrieving {base_url}")
    exit()

main_soup = BeautifulSoup(response.content, "html.parser")

# Extract all rows from the table
rows = main_soup.find_all("tr")

# Iterate over each row to find season data
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 4:
        # Extract the 'Abschlusstabelle' link
        tabelle_link = cells[1].find("a", href=True)
        if tabelle_link:
            relative_url = tabelle_link['href']
            # Construct the full URL for the 'Abschlusstabelle' page
            season_url = "http://www.bulibox.de/abschlusstabellen/" + relative_url
            print(f"Scraping: {season_url}")
            
            season_response = requests.get(season_url)
            if season_response.status_code != 200:
                print(f"Error retrieving {season_url}")
                continue

            season_soup = BeautifulSoup(season_response.content, "html.parser")
            table = season_soup.find("table", class_="abschluss")
            if not table:
                print(f"No table found at {season_url}")
                continue

            # Parse table header
            header_row = table.find("tr")
            headers = [th.get_text(strip=True) for th in header_row.find_all("th")]
            
            # Parse table rows
            rows = []
            for tr in table.find_all("tr")[1:]:
                cells = tr.find_all("td")
                row = [cell.get_text(strip=True) for cell in cells]
                rows.append(row)
            
            # Create DataFrame from the table data
            df_table = pd.DataFrame(rows, columns=headers)
            
            # Extract season from the link's title attribute, e.g., "Abschlusstabelle 2023/2024"
            season_text = tabelle_link.get('title', 'Unknown Season')
            season = season_text.replace("Abschlusstabelle", "").strip()
            # Replace slashes with dashes to avoid directory issues (e.g., "2023/2024" -> "2023-2024")
            season = season.replace("/", "-")
            
            # Build output path and save DataFrame as CSV
            output_path = os.path.join(output_dir, f"{season}.csv")
            df_table.to_csv(output_path, index=False)
            print(f"Saved table for season {season} to {output_path}")
            
            # Wait 5 seconds before the next request
            time.sleep(5)

Scraping: http://www.bulibox.de/abschlusstabellen/B200250.html
Saved table for season 2023-2024 to data/second/abschlusstabellen\2023-2024.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B200249.html
Saved table for season 2022-2023 to data/second/abschlusstabellen\2022-2023.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B200248.html
Saved table for season 2021-2022 to data/second/abschlusstabellen\2021-2022.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B200247.html
Saved table for season 2020-2021 to data/second/abschlusstabellen\2020-2021.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B200246.html
Saved table for season 2019-2020 to data/second/abschlusstabellen\2019-2020.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B200245.html
Saved table for season 2018-2019 to data/second/abschlusstabellen\2018-2019.csv
Scraping: http://www.bulibox.de/abschlusstabellen/B200244.html
Saved table for season 2017-2018 to data/second/abschlusstabellen\2017-2018.csv